In [2]:
import torch
import tensorflow as tf

import tensorflow
import numpy as np

In [3]:
import torch.nn as nn
import torch.nn.functional as F

In [4]:
from tensorflow.keras.layers import Dense, Dropout, Conv3D, LayerNormalization, GlobalAveragePooling1D

In [5]:
!pip install timm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 431 kB 7.4 MB/s 


In [6]:
from timm.models.layers import DropPath, trunc_normal_

Declaring tensor with random values

In [7]:
x_pt = torch.randint(255,(10,8,32,32,3))
x_np = x_pt.numpy()
x_tf = tf.convert_to_tensor(x_np)

x_pt.shape , x_tf.shape

(torch.Size([10, 8, 32, 32, 3]), TensorShape([10, 8, 32, 32, 3]))

## Window Partition Function

In [8]:
def get_window_size(x_size, window_size, shift_size=None):
    use_window_size = list(window_size)
    if shift_size is not None:
        use_shift_size = list(shift_size)
    for i in range(len(x_size)):
        if x_size[i] <= window_size[i]:
            use_window_size[i] = x_size[i]
            if shift_size is not None:
                use_shift_size[i] = 0

    if shift_size is None:
        return tuple(use_window_size)
    else:
        return tuple(use_window_size), tuple(use_shift_size)

In [9]:
window_size = (4,4,4)

B, D, H, W, C = x_pt.shape
window_size = get_window_size((D, H, W), window_size)
window_size

(4, 4, 4)

### PyTorch Window Partition

In [10]:
from operator import mul 
from functools import reduce


In [11]:
def window_partition_pt(x, window_size):
    """
    Args:
        x: (B, D, H, W, C)
        window_size (tuple[int]): window size
    Returns:
        windows: (B*num_windows, window_size*window_size, C)
    """
    B, D, H, W, C = x.shape
    x = x.view(B, D // window_size[0], window_size[0], H // window_size[1], window_size[1], W // window_size[2], window_size[2], C)
    windows = x.permute(0, 1, 3, 5, 2, 4, 6, 7).contiguous().view(-1, reduce(mul, window_size), C)
    return windows

In [12]:
x_windows_pt = window_partition_pt(x_pt, window_size)
x_windows_pt.shape

torch.Size([1280, 64, 3])

### TF window partition func

In [13]:
from functools import reduce

In [14]:
def window_partition_tf(x, window_size):
    """
    Args:
        x: (B, D, H, W, C)
        window_size (tuple[int]): window size
    Returns:
        windows: (B*num_windows, window_size*window_size, C)
    """
    B, D, H, W, C = x.shape
    x = tf.reshape(x, [B, D // window_size[0], window_size[0], H // window_size[1], window_size[1], W // window_size[2], window_size[2], C])
    windows = tf.reshape(tf.transpose(x, perm=[0, 1, 3, 5, 2, 4, 6, 7]), [-1, reduce((lambda x, y: x * y), window_size), C])                                    
                                               
    return windows

In [15]:
x_windows_tf = window_partition_tf(x_tf, window_size)
x_windows_tf.shape

TensorShape([1280, 64, 3])

Comparision

In [16]:
np.array_equal(x_windows_tf.numpy(), x_windows_pt.numpy())

True

## Patch Merging

PyTorch

In [67]:
weight = []

In [70]:
weight

[]

In [186]:
class PatchMerging_pt(nn.Module):
    """ Patch Merging Layer
    Args:
        dim (int): Number of input channels.
        norm_layer (nn.Module, optional): Normalization layer.  Default: nn.LayerNorm
    """
    def __init__(self, dim, norm_layer=nn.LayerNorm):

        super().__init__()
        self.dim = dim
        self.reduction = nn.Linear(4 * dim, 2 * dim, bias=False)
        self.norm = norm_layer(4 * dim)

        print("pytorch weights\n",self.reduction.weight.data.numpy())


    def weight(self):
        return  self.reduction.weight.data.numpy()
    def forward(self, x):
        """ Forward function.
        Args:
            x: Input feature, tensor size (B, D, H, W, C).
        """
        B, D, H, W, C = x.shape

        # padding
        pad_input = (H % 2 == 1) or (W % 2 == 1)
        if pad_input:
            x = F.pad(x, (0, 0, 0, W % 2, 0, H % 2))

        x0 = x[:, :, 0::2, 0::2, :]  # B D H/2 W/2 C
        x1 = x[:, :, 1::2, 0::2, :]  # B D H/2 W/2 C
        x2 = x[:, :, 0::2, 1::2, :]  # B D H/2 W/2 C
        x3 = x[:, :, 1::2, 1::2, :]  # B D H/2 W/2 C
        x = torch.cat([x0, x1, x2, x3], -1)  # B D H/2 W/2 4*C

        x = self.norm(x)
        x = self.reduction(x)

        return x

TF

In [ ]:
from tensorflow.keras.layers import Dense, LayerNormalization, Normalization 

In [187]:
class PatchMerging_tf(tf.keras.layers.Layer):
    def __init__(self, dim,  weight, norm_layer=LayerNormalization):
        super().__init__()
        self.weight = weight
        self.dim = dim
        self.reduction = Dense(2 * dim, use_bias=False,  activation=None)
        self.norm = norm_layer(epsilon=1e-5)
        
    def set_weight(self):
        self.reduction.set_weights([np.transpose(self.weight)])
        print("\n\ntf weights\n", np.transpose(self.reduction.get_weights()[0]))

    # def build(self, input_shapes):
    #     self.reduction.set_weights([np.transpose(self.weight)])

    def call(self, x):
        B, D, H, W, C = x.shape

        # padding
        pad_input = (H % 2 == 1) or (W % 2 == 1)
        if pad_input:
            x = tf.pad(x_tf, [[0,0], [0,0], [0,H%2], [0,W%2], [0,0]])

        x0 = x[:, :, 0::2, 0::2, :]  # B D H/2 W/2 C
        x1 = x[:, :, 1::2, 0::2, :]  # B D H/2 W/2 C
        x2 = x[:, :, 0::2, 1::2, :]  # B D H/2 W/2 C
        x3 = x[:, :, 1::2, 1::2, :]  # B D H/2 W/2 C

        x = tf.concat([x0, x1, x2, x3], axis=-1) # B D H/2 W/2 4*C

        x = self.norm(x)
        x = self.reduction(x)
        
        return x

In [172]:
x_pt = torch.rand(10,8,32,32,3)
x_np = x_pt.numpy()
x_tf = tf.convert_to_tensor(x_np)

In [207]:
patchMerging_pt = PatchMerging_pt(3)

patchMerging_tf = PatchMerging_tf(3, weight=patchMerging_pt.weight())

# weight initialization 
x_tf_merge = patchMerging_tf(x_tf) 
patchMerging_tf.set_weight()


x_pt_merge = patchMerging_pt(x_pt) 
x_tf_merge = patchMerging_tf(x_tf) 

pytorch weights
 [[ 0.13929461 -0.23796813  0.18664165  0.17872916 -0.1861893   0.07879138
   0.13968562  0.2091967  -0.22083881  0.2522571  -0.21668406 -0.1305449 ]
 [-0.10839953 -0.1357809  -0.01730531 -0.0049873   0.22164637  0.25438306
  -0.14403766 -0.18755934 -0.12134344 -0.01897354  0.12711738 -0.23801711]
 [-0.18409483 -0.2080239   0.10796589  0.09955563  0.10302381 -0.00120286
  -0.04078631 -0.23066    -0.10525169 -0.05158307  0.1598327  -0.0034953 ]
 [-0.09502082  0.07829095  0.05060059  0.23620865 -0.05604654  0.12544389
  -0.25806972 -0.07644866  0.28568316 -0.26221994  0.14086194 -0.27208105]
 [ 0.08003189  0.05274725 -0.01824334  0.14429486  0.25617307 -0.24714327
   0.25857413  0.22012089  0.16253045 -0.09206111 -0.27144745 -0.26858944]
 [ 0.21632424 -0.2575141  -0.2543203  -0.27531603 -0.04064408  0.0056396
   0.22927107  0.17797421 -0.22900376  0.19352677 -0.19927354  0.2546929 ]]


tf weights
 [[ 0.13929461 -0.23796813  0.18664165  0.17872916 -0.1861893   0.07879138
 

In [208]:
round = 2
np.array_equal( np.around(x_pt_merge.detach().numpy(), round) , np.around( x_tf_merge.numpy(), round))

True

In [209]:
x_pt_merge.shape, x_tf_merge.shape

(torch.Size([10, 8, 16, 16, 6]), TensorShape([10, 8, 16, 16, 6]))

In [210]:
print( x_pt_merge[0][0][0][1].detach().numpy(),"\n\n\n", x_tf_merge[0][0][0][1].numpy() )

[-0.47216618  0.2653861   0.25594187  0.45342708  0.00656515 -0.80686843] 


 [-0.47216612  0.26538607  0.2559419   0.45342708  0.00656523 -0.80686843]


# Patch Embed 3D

In [48]:
x_pt = torch.rand(10,3,8,32,32)
x_np = x_pt.numpy()
x_tf = tf.convert_to_tensor(x_np)

In [49]:
class PatchEmbed3D_pt(nn.Module):
    """ Video to Patch Embedding.
    Args:
        patch_size (int): Patch token size. Default: (2,4,4).
        in_chans (int): Number of input video channels. Default: 3.
        embed_dim (int): Number of linear projection output channels. Default: 96.
        norm_layer (nn.Module, optional): Normalization layer. Default: None
    """
    def __init__(self, patch_size=(2,4,4), in_chans=3, embed_dim=96, norm_layer=None):
        super().__init__()
        self.patch_size = patch_size

        self.in_chans = in_chans
        self.embed_dim = embed_dim

        self.proj = nn.Conv3d(in_chans, embed_dim, kernel_size=patch_size, stride=patch_size)
        if norm_layer is not None:
            self.norm = norm_layer(embed_dim)
        else:
            self.norm = None

    def forward(self, x):
        """Forward function."""
        # padding
        _, _, D, H, W = x.size()
        if W % self.patch_size[2] != 0:
            x = F.pad(x, (0, self.patch_size[2] - W % self.patch_size[2]))
        if H % self.patch_size[1] != 0:
            x = F.pad(x, (0, 0, 0, self.patch_size[1] - H % self.patch_size[1]))
        if D % self.patch_size[0] != 0:
            x = F.pad(x, (0, 0, 0, 0, 0, self.patch_size[0] - D % self.patch_size[0]))
        x = self.proj(x)  # B C D Wh Ww
        

        if self.norm is not None:
            D, Wh, Ww = x.size(2), x.size(3), x.size(4)
            x = x.flatten(2).transpose(1,2)
            print(x.shape)
            x = self.norm(x)
            print(x.shape)

            x = x.transpose(1, 2).view(-1, self.embed_dim, D, Wh, Ww)

        return x

In [50]:
pathembed = PatchEmbed3D_pt(norm_layer = nn.LayerNorm)
x_em_pt = pathembed(x_pt)
x_em_pt.shape

torch.Size([10, 256, 96])
torch.Size([10, 256, 96])


torch.Size([10, 96, 4, 8, 8])

In [51]:
class PatchEmbed_tf(tf.keras.layers.Layer):
    def __init__(self, patch_size=(2, 4, 4), in_chans=3, embed_dim=96, norm_layer=None):
        super().__init__(name='patch_embed')

        
        self.patch_size = patch_size
        self.in_chans = in_chans
        self.embed_dim = embed_dim

        
        self.proj = Conv3D(embed_dim, kernel_size=patch_size,
                           strides=patch_size, name='proj', data_format = "channels_first")
        
        if norm_layer is not None:
            self.norm = norm_layer(epsilon=1e-5, name='norm')
        else:
            self.norm = None

    def call(self, x):
        B, C, H, W, D = x.get_shape().as_list()
        ## padding
        x = self.proj(x)
        print(x.get_shape())
        if self.norm is not None:
          B, C, D, Wh, Ww = x.get_shape().as_list()
          x = tf.reshape(x, shape=[B, -1, C])
          x = self.norm(x)
          x = tf.reshape(x, shape=[B, C, -1])
          x = tf.reshape(x, shape=[-1, self.embed_dim, D, Wh, Ww])

        return x

In [52]:

patchEmbed_tf = PatchEmbed_tf(norm_layer = LayerNormalization)
x_em_tf = patchEmbed_tf(x_tf)
x_em_tf.get_shape()

(10, 96, 4, 8, 8)


TensorShape([10, 96, 4, 8, 8])

In [ ]:
np.array_equal(x_em_pt.detach().numpy(), x_em_tf.numpy())

False

In [53]:
x_em_pt[0][0][0][0]

tensor([ 0.0364, -0.0411,  0.5812,  0.1997, -0.3046,  0.3152,  0.5739,  0.1697],
       grad_fn=<SelectBackward0>)

In [54]:
x_em_tf[0][0][0][0]

<tf.Tensor: shape=(8,), dtype=float32, numpy=
array([-0.83596694, -0.39291137,  0.5411468 , -0.38081288, -0.69816816,
        0.91106415, -1.7238436 , -0.8586185 ], dtype=float32)>

# MLP

In [55]:
class Mlp_pt(nn.Module):
    """ Multilayer perceptron."""

    def __init__(self, in_features, hidden_features=None, out_features=None, act_layer=nn.GELU, drop=0.):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.act = act_layer()
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.drop = nn.Dropout(drop)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x


In [56]:
class Mlp_tf(tf.keras.layers.Layer):
    def __init__(self, in_features, hidden_features=None, out_features=None, drop=0.):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        self.fc1 = Dense(hidden_features)
        self.fc2 = Dense(out_features)
        self.drop = Dropout(drop)

    def call(self, x):
        x = self.fc1(x)
        x = tf.keras.activations.gelu(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x

# window reverse

In [57]:
def window_reverse_pt(windows, window_size, B, D, H, W):
    """
    Args:
        windows: (B*num_windows, window_size, window_size, C)
        window_size (tuple[int]): Window size
        H (int): Height of image
        W (int): Width of image
    Returns:
        x: (B, D, H, W, C)
    """
    x = windows.view(B, D // window_size[0], H // window_size[1], W // window_size[2], window_size[0], window_size[1], window_size[2], -1)
    x = x.permute(0, 1, 4, 2, 5, 3, 6, 7).contiguous().view(B, D, H, W, -1)
    return 

In [58]:
def window_reverse_tf(windows, window_size, H, W, C):
    x = tf.reshape(B, shape=[ D // window_size[0], H // window_size[1], W // window_size[2], window_size[0], window_size[1], window_size[2], -1])
    x = tf.transpose(x, perm=[0, 1, 4, 2, 5, 3, 6, 7])
    x = tf.reshape(x, shape=[B, D, H, W, -1])
    return x

# WindowAttention3D

In [45]:
x_pt = torch.rand(100,8,32,32,3)
x_np = x_pt.numpy()
x_tf = tf.convert_to_tensor(x_np)

x_pt.shape , x_tf.shape

(torch.Size([100, 8, 32, 32, 3]), TensorShape([100, 8, 32, 32, 3]))

In [33]:
class WindowAttention3D_pt(nn.Module):
    """ Window based multi-head self attention (W-MSA) module with relative position bias.
    It supports both of shifted and non-shifted window.
    Args:
        dim (int): Number of input channels.
        window_size (tuple[int]): The temporal length, height and width of the window.
        num_heads (int): Number of attention heads.
        qkv_bias (bool, optional):  If True, add a learnable bias to query, key, value. Default: True
        qk_scale (float | None, optional): Override default qk scale of head_dim ** -0.5 if set
        attn_drop (float, optional): Dropout ratio of attention weight. Default: 0.0
        proj_drop (float, optional): Dropout ratio of output. Default: 0.0
    """

    def __init__(self, dim, window_size, num_heads, qkv_bias=False, qk_scale=None, attn_drop=0., proj_drop=0.):

        super().__init__()
        self.dim = dim
        self.window_size = window_size  # Wd, Wh, Ww
        self.num_heads = num_heads
        head_dim = dim // num_heads
        self.scale = qk_scale or head_dim ** -0.5

        # define a parameter table of relative position bias
        self.relative_position_bias_table = nn.Parameter(
            torch.zeros((2 * window_size[0] - 1) * (2 * window_size[1] - 1) * (2 * window_size[2] - 1), num_heads))  # 2*Wd-1 * 2*Wh-1 * 2*Ww-1, nH

        # get pair-wise relative position index for each token inside the window
        coords_d = torch.arange(self.window_size[0])
        coords_h = torch.arange(self.window_size[1])
        coords_w = torch.arange(self.window_size[2])
        coords = torch.stack(torch.meshgrid(coords_d, coords_h, coords_w))  # 3, Wd, Wh, Ww
        coords_flatten = torch.flatten(coords, 1)  # 3, Wd*Wh*Ww
        relative_coords = coords_flatten[:, :, None] - coords_flatten[:, None, :]  # 3, Wd*Wh*Ww, Wd*Wh*Ww
        relative_coords = relative_coords.permute(1, 2, 0).contiguous()  # Wd*Wh*Ww, Wd*Wh*Ww, 3
        relative_coords[:, :, 0] += self.window_size[0] - 1  # shift to start from 0
        relative_coords[:, :, 1] += self.window_size[1] - 1
        relative_coords[:, :, 2] += self.window_size[2] - 1

        relative_coords[:, :, 0] *= (2 * self.window_size[1] - 1) * (2 * self.window_size[2] - 1)
        relative_coords[:, :, 1] *= (2 * self.window_size[2] - 1)
        relative_position_index = relative_coords.sum(-1)  # Wd*Wh*Ww, Wd*Wh*Ww
        self.register_buffer("relative_position_index", relative_position_index)

        self.qkv = nn.Linear(dim, dim * 3, bias=qkv_bias)
        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)

        trunc_normal_(self.relative_position_bias_table, std=.02)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x, mask=None):
        """ Forward function.
        Args:
            x: input features with shape of (num_windows*B, N, C)
            mask: (0/-inf) mask with shape of (num_windows, N, N) or None
        """
        B_, N, C = x.shape
        qkv = self.qkv(x).reshape(B_, N, 3, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
        q, k, v = qkv[0], qkv[1], qkv[2]  # B_, nH, N, C

        q = q * self.scale
        attn = q @ k.transpose(-2, -1)

        relative_position_bias = self.relative_position_bias_table[self.relative_position_index[:N, :N].reshape(-1)].reshape(
            N, N, -1)  # Wd*Wh*Ww,Wd*Wh*Ww,nH
        relative_position_bias = relative_position_bias.permute(2, 0, 1).contiguous()  # nH, Wd*Wh*Ww, Wd*Wh*Ww
        attn = attn + relative_position_bias.unsqueeze(0) # B_, nH, N, N

        if mask is not None:
            nW = mask.shape[0]
            attn = attn.view(B_ // nW, nW, self.num_heads, N, N) + mask.unsqueeze(1).unsqueeze(0)
            attn = attn.view(-1, self.num_heads, N, N)
            attn = self.softmax(attn)
        else:
            attn = self.softmax(attn)

        attn = self.attn_drop(attn)

        x = (attn @ v).transpose(1, 2).reshape(B_, N, C)
        x = self.proj(x)
        x = self.proj_drop(x)
        return x

In [29]:
class WindowAttention3D_tf(tf.keras.layers.Layer):
    def __init__(self, dim, window_size, num_heads, qkv_bias=True, qk_scale=None, attn_drop=0., proj_drop=0.):
        super().__init__()
        self.dim = dim
        self.window_size = window_size
        self.num_heads = num_heads
        head_dim = dim // num_heads
        self.scale = qk_scale or head_dim ** -0.5


        self.qkv = Dense(dim * 3, use_bias=qkv_bias)
        self.attn_drop = Dropout(attn_drop)
        self.proj = Dense(dim)
        self.proj_drop = Dropout(proj_drop)


    def build(self, input_shape):
        self.relative_position_bias_table = self.add_weight(shape=((2 * self.window_size[0] - 1) * (2 * self.window_size[1] - 1)* (2 * window_size[2] - 1), self.num_heads),
                                                            initializer=tf.initializers.Zeros(), trainable=True) # 2*Wd-1 * 2*Wh-1 * 2*Ww-1, nH

        coords_d = np.arange(self.window_size[0])
        coords_h = np.arange(self.window_size[1])
        coords_w = np.arange(self.window_size[2])
        coords = np.stack(np.meshgrid(coords_d, coords_h, coords_w, indexing='ij')) # 3, Wd, Wh, Ww
        coords_flatten = coords.reshape(3, -1)
        relative_coords = coords_flatten[:, :,
                                         None] - coords_flatten[:, None, :]
        relative_coords = relative_coords.transpose([1, 2, 0])
        relative_coords[:, :, 0] += self.window_size[0] - 1
        relative_coords[:, :, 1] += self.window_size[1] - 1
        relative_coords[:, :, 2] += self.window_size[2] - 1

        relative_coords[:, :, 0] *= (2 * self.window_size[1] - 1) * (2 * self.window_size[2] - 1)
        relative_coords[:, :, 1] *= (2 * self.window_size[2] - 1)
        relative_position_index = relative_coords.sum(-1).astype(np.int64)
        

        self.relative_position_index = tf.Variable(initial_value=tf.convert_to_tensor(
            relative_position_index), trainable=False)
        self.built = True


    def call(self, x, mask=None):
        B_, N, C = x.get_shape().as_list()
        qkv = tf.transpose(tf.reshape(self.qkv(
            x), shape=[-1, N, 3, self.num_heads, C // self.num_heads]), perm=[2, 0, 3, 1, 4])
        q, k, v = qkv[0], qkv[1], qkv[2]

        q = q * self.scale
        attn = (q @ tf.transpose(k, perm=[0, 1, 3, 2]))
        index = tf.reshape(self.relative_position_index[:N, :N], shape=[-1])
        relative_position_bias = tf.gather(self.relative_position_bias_table, index)
        relative_position_bias = tf.reshape(relative_position_bias, shape=[N, N, -1])
        relative_position_bias = tf.transpose(relative_position_bias, perm=[2, 0, 1])
        attn = attn + tf.expand_dims(relative_position_bias, axis=0)

        if mask is not None:
            nW = mask.get_shape()[0]  # tf.shape(mask)[0]
            attn = tf.reshape(attn, shape=[-1, nW, self.num_heads, N, N]) + tf.cast(
                tf.expand_dims(tf.expand_dims(mask, axis=1), axis=0), attn.dtype)
            attn = tf.reshape(attn, shape=[-1, self.num_heads, N, N])
            attn = tf.nn.softmax(attn, axis=-1)
        else:
            attn = tf.nn.softmax(attn, axis=-1)

        attn = self.attn_drop(attn)

        x = tf.transpose((attn @ v), perm=[0, 2, 1, 3])
        x = tf.reshape(x, shape=[-1, N, C])
        x = self.proj(x)
        x = self.proj_drop(x)
        return x

In [41]:
atten_tf = WindowAttention3D_tf(3, window_size=(2,7, 7), num_heads=3)
wp_tf = window_partition_tf(x_tf, (4,4,4))
a_tf = atten_tf(wp_tf)
a_tf.get_shape()

TensorShape([12800, 64, 3])

In [42]:
attention_pt = WindowAttention3D_pt(3, window_size=(2,7, 7), num_heads=3)
wp_pt = window_partition_pt(x_pt, (4,4,4))
a_pt = attention_pt(s)
a_pt.shape

torch.Size([12800, 64, 3])

In [169]:
a_tf[0][0]

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([-0.3371957 , -0.07501692, -0.037497  ], dtype=float32)>

In [170]:
a_pt[0][0]

tensor([0.1801, 0.2295, 0.7769], grad_fn=<SelectBackward0>)